# Revised case normalization for Hirslanden Salem 2018

Notes:




In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx'),
                     'Hirslanden Salem-Spital', '2018', 'Änderungen _SA_2018')
print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx', hospital_name_db='Hirslanden Salem-Spital', year='2018', sheet='Änderungen _SA_2018')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-21 09:03:43.693 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:38 - Read 46 cases for Hirslanden Salem-Spital 2018
2022-11-21 09:03:43.697 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:73 - TYPES:
datum                             string
tranche                           string
case_id                           object
patient_id                        object
gender                            string
age_years                          int64
bfs_code                          string
duration_of_stay                   int64
pflegetage neu                    string
pccl                               int64
pccl neu                          string
old_pd                            string
primary_diagnosis                 string
added_icds                        string
removed_icds                      string
added_chops                       string
removed_chops                     string
drg                      

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,0041602986,90C3E16484AD653A,M,88,12,41602986,M171,M171,M200,3,I43B,[N183],[],[],[]
1,0041637726,D774DF60034ACAA4,W,53,19,41637726,T842,T842,M200,3,I09C,"[K564, T811]",[],[],[]
2,0041641561,4A03EA9845D13800,M,81,9,41641561,N185,N1793,M100,3,L60C,[],[],[],[]
3,0041643625,E3CD9E9C233B926F,M,83,10,41643625,E1172,E1172,M100,3,F13C,[M8617],[M8697],[],[]
4,0041644263,80BD3429C23B3A1E,W,76,10,41644263,A4152,A418,M200,3,T60E,"[J9600, B962]",[J9699],[],[]
5,0041649868,AECB468A426E92CC,M,88,14,41649868,S7208,S7208,M200,3,I46C,"[I360, I5001, R31]",[],[],[]
6,0041656883,610A5F0682A8B204,W,86,13,41656883,J100,J100,M100,3,E77E,[J9600],[],[],[]
7,0041683635,44AB9518C38C5FD0,M,88,8,41683635,S723,S723,M200,3,I08D,"[L8914, D6833]",[],[],[]
8,0041689417,AF8052F234B54DF7,W,86,3,41689417,N328,N328,M200,3,L18B,"[F009, G309]",[F03],[],[]
9,0041695908,E459BCEBEA8AD752,W,86,7,41695908,J209,J209,M100,3,E69C,[I5013],[I5019],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases

2022-11-21 09:03:44.991 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:95 - Read 9686 rows from the DB, for the hospital 'Hirslanden Salem-Spital' in 2018


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,476343,0041788723,J181,"[J91, N183, K590, I489, I480, ...]",99221A::20180614,"[9962::20180614, 948X40::20180614, 8965::20180614]",W,77,0,0,15,0,01,2018-06-14,0,00,2018-06-29
1,476804,0041679610,M706,"[M7965, M160, J101, I1190, Z950, ...]",,[],M,87,0,0,17,0,01,2018-02-12,0,00,2018-03-01
2,476884,0041711681,I7024,"[B957, I724, N183, D648, I1090, ...]",8411:L:20180320,"[990410::20180313, 395021:L:20180318, 399019:L:20180318, 395019:L:20180318, 0041:L:20180318, ...]",W,88,0,0,22,0,01,2018-03-13,0,00,2018-04-04
3,477209,0041691601,M4316,"[M9963, I1090, N183, G4731, J459, ...]",8108::20180308,"[816210::20180308, 785035::20180308, 030991::20180308, 030934::20180308, 009920::20180308, ...]",W,66,0,0,9,0,01,2018-03-07,0,00,2018-03-16
4,477233,0041683635,S723,"[X599, Z9664, D62, F051, E1190, ...]",793512:R:20180216,"[990A::20180215, 009910::20180216, 990410::20180216]",M,88,0,0,8,0,06,2018-02-15,0,00,2018-02-23
5,477598,0041779228,S801,"[X599, Z922, R600, I1090, I420, ...]",86421C:R:20180605,[],M,87,0,0,17,0,01,2018-06-04,0,00,2018-06-21
6,477799,0041695908,J209,"[J9609, I1100, I1090, E784, N1791, ...]",883841:B:20180227,[8965::20180227],W,86,0,0,7,0,01,2018-02-27,0,00,2018-03-06
7,477822,0041779882,J4409,"[B978, J9601, N1791, N3942, I2519, ...]",992216::20180602,[],W,97,0,0,6,0,01,2018-06-02,0,00,2018-06-08
8,477966,0041742275,M2359,"[T932, M1046, M8166, M6586, Z861, ...]",815422:R:20180423,"[807620:R:20180423, 803630::20180423, 819231:R:20180423, 992909::20180423, 009940::20180423]",M,72,0,0,6,0,01,2018-04-22,0,00,2018-04-28
9,478146,0041869375,T846,"[Y828, B957, M512, I1090, E789, ...]",862A1E::20181006,"[889714::20181004, 030991::20181006, 805910::20181006, 832111::20181006, 030934::20181006, ...]",M,65,0,0,22,0,01,2018-10-03,0,00,2018-10-25


In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-21 09:03:45.538 | INFO     | src.revised_case_normalization.notebook_functions.group:group:86 - Grouping 46 cases ...
2022-11-21 09:03:46.623 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:91 - Grouped 46 cases into: 46 revisions, 479 diagnoses rows, 192 procedure rows


In [7]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,476343,E77C,1.13,1.13,4,2022-12-31
1,476804,I72Z,1.07,1.07,3,2022-12-31
2,476884,F59B,2.55,2.55,4,2022-12-31
3,477209,I09D,2.69,2.69,3,2022-12-31
4,477233,I08C,2.32,2.32,4,2022-12-31
5,477598,X06A,2.83,2.83,4,2022-12-31
6,477799,E69A,0.93,0.93,4,2022-12-31
7,477822,E65C,0.67,0.67,3,2022-12-31
8,477966,I43A,2.68,2.68,4,2022-12-31
9,478146,I12B,2.51,2.51,3,2022-12-31


In [8]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-21 09:03:46.645 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:259 - Trying to insert 46 cases into the 'Revisions' table ...
2022-11-21 09:03:50.830 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:296 - Inserted 46 cases into the 'Revisions' table
2022-11-21 09:03:50.833 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:309 - Trying to insert 479 rows into the 'Diagnoses' table ...
2022-11-21 09:04:01.667 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:347 - Inserted 479 rows into the 'Diagnoses' table
2022-11-21 09:04:01.671 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:358 - Trying to insert 192 rows into the 'Procedures' table ...
2022-11-21 09:04:03.364 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:404 - Inserted 192 rows into the 'Procedures' table
